To-Do:

Datasets:
etools.ch --> Results List
ThaiCert --> Description / Tools, Links Harvest to Wordcloud
ATT&CK --> TTPS
Twitter
Wordcloud
OTX (Maybe)
Zotero --> https://github.com/sailist/pyzotero-local
YT


## Setup

### Install Packages

In [56]:
%%capture

import sys

!{sys.executable} -m pip install stix2
!{sys.executable} -m pip install pandas

### ThaiCert

In [17]:
import requests
import json

url_thaicert = "https://apt.thaicert.or.th/cgi-bin/getcard.cgi?g=all&o=j"

def pull_thaicert_db():
    http_req = requests.get(url_thaicert).json()
    
    with open('./database/thaicert.json', 'w+') as output:
        output.write(json.dumps(http_req, indent=2))
        print("\n[+] Database Saved !")
    
pull_thaicert_db()


[+] Database Saved !


### MITRE ATT&CK

In [7]:
from stix2 import MemoryStore

def get_data_from_branch(domain):
    """get the ATT&CK STIX data from MITRE/CTI. Domain should be 'enterprise-attack', 'mobile-attack' or 'ics-attack'. Branch should typically be master."""
    stix_json = requests.get(f"https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/{domain}/{domain}.json").json()
    return MemoryStore(stix_data=stix_json["objects"])

src = get_data_from_branch("enterprise-attack")

## Search Datasets

### Search APT in ThaiCert

Replace search_apt with APT name

In [69]:
search_apt='Emissary Panda'

print("Actor:")
!cat database/thaicert.json | jq '.values[] | select(.names[].name=="'"$search_apt"'") | .actor'
print()

print("Other Names:")
!cat database/thaicert.json | jq '.values[] | select(.names[].name=="'"$search_apt"'") | .names[].name'
print()

print("Country:")
!cat database/thaicert.json | jq '.values[] | select(.names[].name=="'"$search_apt"'") | .country[]'
print()


print("Description:")
!cat database/thaicert.json | jq -r '.values[] | select(.names[].name=="'"$search_apt"'") | .description'
print()

print("OSINT Links:")
links = !cat database/thaicert.json | jq '.values[] | select(.names[].name=="'"$search_apt"'") | .information[]'
for link in links:
    print(link)

Actor:
"Emissary Panda, APT 27, LuckyMouse, Bronze Union"

Other Names:
"Emissary Panda"
"APT 27"
"LuckyMouse"
"Bronze Union"
"TG-3390"
"TEMP.Hippo"
"Budworm"
"Group 35"
"ATK 15"
"Iron Tiger"
"Earth Smilodon"
"ZipToken"

Country:
"China"

Description:
Threat Group-3390 is a Chinese threat group that has extensively used strategic Web compromises to target victims. The group has been active since at least 2010 and has targeted organizations in the aerospace, government, defense, technology, energy, and manufacturing sectors.

Emissary Panda has some overlap with {{Turbine Panda, APT 26, Shell Crew, WebMasters, KungFu Kittens}} and possibly {{UNC215}}.

This actor worked together with {{TA428}} in Operation StealthyTrident.

OSINT Links:
"https://www.secureworks.com/research/threat-group-3390-targets-organizations-for-cyberespionage"
"https://www.secureworks.com/research/a-peek-into-bronze-unions-toolbox"
"https://www.secureworks.com/research/bronze-union"


### Search APT in ATT&CK

Get data available on given APT group.

In [33]:
from stix2 import Filter

def get_group_by_alias(thesrc, alias):
    return thesrc.query([
        Filter('type', '=', 'intrusion-set'),
        Filter('aliases', '=', alias)
    ])[0]

apt_data = get_group_by_alias(src, 'Emissary Panda')

apt_intrusion_set = src.get(apt_data.id)

In [55]:
print("Description:")
apt_data.description


Description:


'[Threat Group-3390](https://attack.mitre.org/groups/G0027) is a Chinese threat group that has extensively used strategic Web compromises to target victims. (Citation: Dell TG-3390) The group has been active since at least 2010 and has targeted organizations in the aerospace, government, defense, technology, energy, and manufacturing sectors. (Citation: SecureWorks BRONZE UNION June 2017) (Citation: Securelist LuckyMouse June 2018)'

In [59]:
from stix2.utils import get_type_from_id
import pandas as pd

def get_techniques_by_group_software(thesrc, group_stix_id):
    # get the malware, tools that the group uses
    group_uses = [
        r for r in thesrc.relationships(group_stix_id, 'uses', source_only=True)
        if get_type_from_id(r.target_ref) in ['malware', 'tool']
    ]
    print(group_uses)
    # get the technique stix ids that the malware, tools use
    software_uses = thesrc.query([
        Filter('type', '=', 'relationship'),
        Filter('relationship_type', '=', 'uses'),
        Filter('source_ref', 'in', [r.source_ref for r in group_uses])
    ])

    #get the techniques themselves
    return thesrc.query([
        Filter('type', '=', 'attack-pattern'),
        Filter('id', 'in', [r.target_ref for r in software_uses])
    ])

data_df = []
apt_techniques = get_techniques_by_group_software(src, apt_intrusion_set)
for technique in apt_techniques:
    row = []
    row.append(technique.external_references[0].external_id)
    row.append(technique.name)
    data_df.append(row)
pd.DataFrame(data_df, columns=['ATT&CK Name', 'ATT&CK ID'])

[Relationship(type='relationship', spec_version='2.1', id='relationship--614c18a5-2cee-48ac-898d-e1b85a91e44d', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2017-05-31T21:33:27.063Z', modified='2020-03-31T14:49:39.188Z', relationship_type='uses', description='(Citation: Dell TG-3390)(Citation: SecureWorks BRONZE UNION June 2017)', source_ref='intrusion-set--fb366179-766c-4a4a-afa1-52bff1fd601c', target_ref='malware--a60657fa-e2e7-4f8f-8128-a882534ae8c5', revoked=False, external_references=[ExternalReference(source_name='Dell TG-3390', description='Dell SecureWorks Counter Threat Unit Threat Intelligence. (2015, August 5). Threat Group-3390 Targets Organizations for Cyberespionage. Retrieved August 18, 2018.', url='https://www.secureworks.com/research/threat-group-3390-targets-organizations-for-cyberespionage'), ExternalReference(source_name='SecureWorks BRONZE UNION June 2017', description='Counter Threat Unit Research Team. (2017, June 27). BRONZE UNION Cy

,ATT&CK Name,ATT&CK ID
0,T1560.002,Archive via Library
1,T1053.002,At (Windows)
2,T1119,Automated Collection
3,T1548.002,Bypass User Account Control
4,T1043,Commonly Used Port
5,T1574.001,DLL Search Order Hijacking
6,T1574.002,DLL Side-Loading
7,T1002,Data Compressed
8,T1030,Data Transfer Size Limits
9,T1005,Data from Local System


In [62]:
src.get("malware--a60657fa-e2e7-4f8f-8128-a882534ae8c5")

Malware(type='malware', spec_version='2.1', id='malware--a60657fa-e2e7-4f8f-8128-a882534ae8c5', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2017-05-31T21:32:47.412Z', modified='2021-06-17T19:03:17.306Z', name='OwaAuth', description='[OwaAuth](https://attack.mitre.org/software/S0072) is a Web shell and credential stealer deployed to Microsoft Exchange servers that appears to be exclusively used by [Threat Group-3390](https://attack.mitre.org/groups/G0027). (Citation: Dell TG-3390)', is_family=True, revoked=False, external_references=[ExternalReference(source_name='mitre-attack', url='https://attack.mitre.org/software/S0072', external_id='S0072'), ExternalReference(source_name='Dell TG-3390', description='Dell SecureWorks Counter Threat Unit Threat Intelligence. (2015, August 5). Threat Group-3390 Targets Organizations for Cyberespionage. Retrieved August 18, 2018.', url='https://www.secureworks.com/research/threat-group-3390-targets-organizations-for-cybere